In [49]:
import sys
import os

# 현재 디렉토리의 상위 폴더 경로를 구합니다.
# matplotsoccer를 설치(pip)해서 라이브러리르 사용할 수 있지만, 본 연구에서는 좀 더 시각적으로 설명하기 위해 라이브러리를 조금 수정해보았다
parent_dir = os.path.dirname(os.getcwd())

# 상위 폴더 경로를 sys.path에 추가합니다.
sys.path.insert(0, parent_dir)

In [50]:
import os
import warnings
import tqdm
import pandas as pd
import numpy as np
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [51]:
%load_ext autoreload
%autoreload 2
import socceraction.vaep.features as fs
import socceraction.vaep.labels as lab

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
spadl_datafolder = "../data-fifa/La Liga/spadl-statsbomb/"
feature_datafolder = "../data-fifa/La Liga/feature/"
label_datafoler = "../data-fifa/La Liga/label/"


train_spadl_h5 = os.path.join(spadl_datafolder, "train_competitions.h5")
train_features_h5 = os.path.join(feature_datafolder, "train_features.h5")
train_labels_h5 = os.path.join(label_datafoler, "train_labels.h5")

valid_spadl_h5 = os.path.join(spadl_datafolder, "valid_competitions.h5")
valid_features_h5 = os.path.join(feature_datafolder, "valid_features.h5")
valid_labels_h5 = os.path.join(label_datafoler, "valid_labels.h5")

test_spadl_h5 = os.path.join(spadl_datafolder, "test_competitions.h5")
test_features_h5 = os.path.join(feature_datafolder, "test_features.h5")
test_labels_h5 = os.path.join(label_datafoler, "test_labels.h5")

In [53]:
train_games = pd.read_hdf(train_spadl_h5, "games")
print("nb of games:", len(train_games))

valid_games = pd.read_hdf(valid_spadl_h5, "games")
print("nb of games:", len(valid_games))

test_games = pd.read_hdf(test_spadl_h5, "games")
print("nb of games:", len(test_games))

nb of games: 800
nb of games: 33
nb of games: 35


In [54]:
#Select feature set X

xfns = [
    fs.actiontype,
    fs.actiontype_onehot,
    
    #foot, head, head/other, other
    #foot중 왼/오른발 구분은 bodypart에서 하지 않음
    fs.bodypart,
    fs.bodypart_onehot,
    
    #각 수행한 액션의 왼/ 오른발로 구분함
    # fs.bodypart_detailed,
    # fs.bodypart_detailed_onehot,
    
    fs.result,
    fs.result_onehot,
    
    #goalscore_team, goalscore_opponent, goalscore_diff 3개의 feature를 불러옴
    fs.goalscore,
    
    #action type and result사이의 원핫인코딩값?
    # fs.actiontype_result_onehot,
    
    
    #action의 시작, 끝 위치
    fs.startlocation,
    fs.endlocation,
    
    #movement는 x좌표변화량, y좌표변화량
    fs.movement,
    
    #space_delta는 (x,y)좌표의 변화량
    fs.space_delta,
    
    #startlocation, endloaction에서 goal까지 거리랑 각도
    fs.startpolar,
    fs.endpolar,
    
    #home인지 away인지
    fs.team,
    
    #time : 3가지 feature가 나옴
    #1.period_id = 전반1 / 후반2
    #2.time_seconds = 전/후반의 시작 후에 time
    #3.time_seconds_overall = 게임시작후 초
    # fs.time,
    
    #time_delta1 = 현 action time_seconds - 전 action time_seconds
    #time_delta2 = 현 action time_seconds - 전전 action time_seconds
    fs.time_delta,
    
]


In [55]:
nb_prev_actions = 3

Xcols = fs.feature_column_names(xfns, nb_prev_actions)
Xcols

['type_id_a0',
 'type_id_a1',
 'type_id_a2',
 'type_pass_a0',
 'type_cross_a0',
 'type_throw_in_a0',
 'type_freekick_crossed_a0',
 'type_freekick_short_a0',
 'type_corner_crossed_a0',
 'type_corner_short_a0',
 'type_take_on_a0',
 'type_foul_a0',
 'type_tackle_a0',
 'type_interception_a0',
 'type_shot_a0',
 'type_shot_penalty_a0',
 'type_shot_freekick_a0',
 'type_keeper_save_a0',
 'type_keeper_claim_a0',
 'type_keeper_punch_a0',
 'type_keeper_pick_up_a0',
 'type_clearance_a0',
 'type_bad_touch_a0',
 'type_non_action_a0',
 'type_dribble_a0',
 'type_goalkick_a0',
 'type_pass_a1',
 'type_cross_a1',
 'type_throw_in_a1',
 'type_freekick_crossed_a1',
 'type_freekick_short_a1',
 'type_corner_crossed_a1',
 'type_corner_short_a1',
 'type_take_on_a1',
 'type_foul_a1',
 'type_tackle_a1',
 'type_interception_a1',
 'type_shot_a1',
 'type_shot_penalty_a1',
 'type_shot_freekick_a1',
 'type_keeper_save_a1',
 'type_keeper_claim_a1',
 'type_keeper_punch_a1',
 'type_keeper_pick_up_a1',
 'type_clearance_a1

In [56]:
def getXY(games, Xcols, features_h5, labels_h5):
    X = []
    for game_id in tqdm.tqdm(games.game_id, desc="Selecting features"):      
        Xi = pd.read_hdf(features_h5, f"game_{game_id}")
        X.append(Xi[Xcols])
    X = pd.concat(X).reset_index(drop=True)

    # 2. Select label Y
    Ycols = ["scores","concedes"]
    Y = []
    for game_id in tqdm.tqdm(games.game_id, desc="Selecting label"):
        Yi = pd.read_hdf(labels_h5, f"game_{game_id}")
        Y.append(Yi[Ycols])
    Y = pd.concat(Y).reset_index(drop=True)
    
    return X, Y

In [57]:
X_train, Y_train = getXY(train_games, Xcols, train_features_h5, train_labels_h5)
X_train.shape, Y_train.shape

Selecting features:  11%|█▏        | 91/800 [00:00<00:06, 104.60it/s]

Selecting label: 100%|██████████| 800/800 [00:03<00:00, 202.68it/s]


((1686557, 154), (1686557, 2))

In [58]:
X_valid, Y_valid = getXY(valid_games, Xcols, valid_features_h5, valid_labels_h5)
X_valid.shape, Y_valid.shape

Selecting features:  33%|███▎      | 11/33 [00:00<00:00, 105.06it/s]

Selecting label: 100%|██████████| 33/33 [00:00<00:00, 171.13it/s]


((75786, 154), (75786, 2))

In [59]:
X_test, Y_test = getXY(test_games,Xcols, test_features_h5, test_labels_h5)
X_test.shape, Y_test.shape

Selecting label: 100%|██████████| 35/35 [00:00<00:00, 180.55it/s]


((82108, 154), (82108, 2))

## -Data preprocessing

- VAEP의 game-state는 이전 state의 정보를 활용함
- 즉, 전반->후반, 후반->연장등으로 period가 넘어가는 부분을 처러하는 코드가 구현되어있지 않음 : SPADL의 단점.
- 저의 연구에서는 SPADL의 feature.py를 따로 처리하는 부분을 수정하지는 않았습니다.

1. time_delta 오류값 처리 <br><br>
- X_train, X_valid, X_test에는 오류값이 들어있는데, 바로 time_delta값이다<br>
- time_delta = 현재 action time - 전 action time <br>
- 위 방식대로 각 경기를 연결하면, 현재 경기의 첫번째 time_delta = 현재 경기 첫 time - 전 경기 마지막 time가 들어가므로 오류가 발생함 => 개별적으로 전처리해줌<br>

In [60]:
count1 = 0
count2 = 0
count3 = 0

#train, valid, test모두 time_delata값에 음수가 들어가는 현상발생
for i,value in enumerate(X_train['time_delta_1']):
    if value<0:
        count1+=1
        
for i,value in enumerate(X_valid['time_delta_1']):
    if value<0:
        count2+=1
        
for i,value in enumerate(X_test['time_delta_1']):
    if value<0:
        count3+=1

count1,count2,count3

(800, 33, 35)

In [61]:
def time_preprocessing(data):
    for i in tqdm.tqdm(range(len(data))):
        if data.loc[i,'time_delta_1'] < 0.0:
            data.loc[i,'time_delta_1'] = 0.0
            
        if data.loc[i,'time_delta_2'] < 0.0:
            data.loc[i,'time_delta_2'] = 0.0    

    return data

In [62]:
X_train = time_preprocessing(X_train)
X_valid = time_preprocessing(X_valid)
X_test = time_preprocessing(X_test)

  1%|▏         | 22127/1686557 [00:00<00:22, 75518.11it/s]

100%|██████████| 82108/82108 [00:01<00:00, 73333.48it/s]


In [63]:
X_train.shape, X_valid.shape,X_test.shape

((1686557, 154), (75786, 154), (82108, 154))

In [64]:
Y_train.shape, Y_valid.shape, Y_test.shape

((1686557, 2), (75786, 2), (82108, 2))

2. 레이블의 개수를 살펴보면, 대부분 scores과 concedes=0인 것을 확인할 수 있음

- 득점/실점이 적은 축구에서 레이블의 불균형 문제 발생

In [65]:
print(Y_train.value_counts(),'\n')
print(Y_valid.value_counts(),'\n')
print(Y_test.value_counts())

scores  concedes
False   False       1662115
True    False         20688
False   True           3754
Name: count, dtype: int64 

scores  concedes
False   False       74776
True    False         858
False   True          152
Name: count, dtype: int64 

scores  concedes
False   False       81005
True    False         961
False   True          142
Name: count, dtype: int64


3. 이기고 있는 상황 / 비기고 있는 상황 / 지고 있는 상황을 categorical feature로 추가

-기존 goalscore=[-8,8]의 수치형 데이터가 존재하는데, 본 연구에서는 추가적인 feature를 사용하기 위해 goalscore를 활용하여 승/패/무의 categorical data로 추가

In [66]:
def categorical_goalscore(X):
    #카테고리로 넣었을 때는, 패무승 중인 경우 3가지로 구분해서 feature를 추가
    #기존 수치형데이터인 득실차는 유지하고, 새로운 카테고리 승/무/패인 상황을 추가함
    cate = "MatchStatus"
    for i, value in tqdm.tqdm(enumerate(X['goalscore_diff']),desc='Losing:0 Drawing:1 Winning:2'):
        if value<0:
            X.loc[i,cate] = 'Losing'
        elif value==0:
            X.loc[i,cate] = 'Drawing'
        else:
            X.loc[i,cate] = 'Winning'
                                       
    return X

In [67]:
#본 연구에서는 성능이 좀 더 떨어지므로 해당 feature는 사용하진 않음
X_train = categorical_goalscore(X_train)
X_valid = categorical_goalscore(X_valid)
X_test = categorical_goalscore(X_test)

Losing:0 Drawing:1 Winning:2: 4292it [00:00, 22081.97it/s]

Losing:0 Drawing:1 Winning:2: 1686557it [01:22, 20404.83it/s]
Losing:0 Drawing:1 Winning:2: 75786it [00:03, 19775.99it/s]
Losing:0 Drawing:1 Winning:2: 82108it [00:04, 20388.46it/s]


In [68]:
X_train.shape, X_valid.shape,X_test.shape

((1686557, 155), (75786, 155), (82108, 155))

4. zone categorical data추가

-기존 position데이터가 존재하지만, 각 위치를 구역(zone, grid)별로 만들어서 categorical data로 사용할 예정

In [69]:
def get_zone_index(x, y):
    zone_width = 108/12  # zone의 너비
    zone_height = 72/8  # zone의 높이

    # x좌표가 속하는 zone의 인덱스 계산
    zone_col = int(x // zone_width)
    if zone_col == 12:  # 경계 체크
        zone_col -= 1

    # y좌표가 속하는 zone의 인덱스 계산
    zone_row = int(y // zone_height)
    if zone_row == 8:  # 경계 체크
        zone_row -= 1

    # zone의 인덱스를 반환
    return zone_row * 12 + zone_col

In [70]:
def location_zone(X):
    start_position_feature = [['start_x_a0','start_y_a0'],['start_x_a1','start_y_a1'], ['start_x_a2','start_y_a2']]
    end_position_features = [['end_x_a0','end_y_a0'], ['end_x_a1','end_y_a1'], ['end_x_a2','end_y_a2']]
    
    for x_col, y_col in tqdm.tqdm(start_position_feature):
        col_name = 'start_zone_' + x_col[-2:]
        X[col_name] = X.apply(lambda row: get_zone_index(row[x_col], row[y_col]), axis=1)
    
    # 종료 위치에 대한 zone feature 추가
    for x_col, y_col in tqdm.tqdm(end_position_features):
        col_name = 'end_zone_' + x_col[-2:]
        X[col_name] = X.apply(lambda row: get_zone_index(row[x_col], row[y_col]), axis=1)

    return X

In [71]:
X_train = location_zone(X_train)
X_valid = location_zone(X_valid)
X_test = location_zone(X_test)

100%|██████████| 3/3 [00:02<00:00,  1.28it/s]


In [72]:
X_train.shape, X_valid.shape,X_test.shape

((1686557, 161), (75786, 161), (82108, 161))

## 이진분류에 사용할 데이터 저장

In [73]:
folder = '../data/binary-data/'

# os.makedirs(folder + 'train/')
# os.makedirs(folder + 'valid/')
# os.makedirs(folder + 'test/')

X_train.to_csv(folder + 'train/X_train.csv', index=False)
Y_train.to_csv(folder + 'train/Y_train.csv', index=False)

X_valid.to_csv(folder + 'valid/X_valid.csv', index=False)
Y_valid.to_csv(folder + 'valid/Y_valid.csv', index=False)

X_test.to_csv(folder + 'test/X_test.csv', index=False)
Y_test.to_csv(folder + 'test/Y_test.csv', index=False)

## 다중 분류를 활용하기 위해 multi_data를 추가 제작

In [74]:
multi_X_train = pd.read_csv(folder + 'train/X_train.csv')
multi_Y_train = pd.read_csv(folder + 'train/Y_train.csv')           

multi_X_valid = pd.read_csv(folder + 'valid/X_valid.csv')
multi_Y_valid = pd.read_csv(folder + 'valid/Y_valid.csv')           

multi_X_test = pd.read_csv(folder + 'test/X_test.csv')
multi_Y_test = pd.read_csv(folder + 'test/Y_test.csv')  

In [75]:
multi_X_train.shape, multi_Y_train.shape, multi_X_valid.shape, multi_Y_valid.shape, multi_X_test.shape, multi_Y_test.shape

((1686557, 161),
 (1686557, 2),
 (75786, 161),
 (75786, 2),
 (82108, 161),
 (82108, 2))

In [76]:
def multi_class(scores, concedes):
    if (not scores) and (not concedes):
        return 0
    elif scores and (not concedes):
        return 1
    elif (not scores) and concedes:
        return 2
    else:
        print("error : ",scores,concedes)
        return np.nan  # 'score=True, concede=True'는 불가능한 조건으로 가정

In [77]:
multi_Y_train = multi_Y_train.apply(lambda row: multi_class(row['scores'], row['concedes']), axis=1)
multi_Y_valid = multi_Y_valid.apply(lambda row: multi_class(row['scores'], row['concedes']), axis=1)
multi_Y_test = multi_Y_test.apply(lambda row: multi_class(row['scores'], row['concedes']), axis=1)

In [78]:
multi_X_train.shape, multi_Y_train.shape, multi_X_valid.shape, multi_Y_valid.shape, multi_X_test.shape, multi_Y_test.shape

((1686557, 161), (1686557,), (75786, 161), (75786,), (82108, 161), (82108,))

In [79]:
folder = '../data/multiclass-data/'

# os.makedirs(folder + 'train/')
# os.makedirs(folder + 'valid/')
# os.makedirs(folder + 'test/')

multi_X_train.to_csv(folder + 'train/X_train.csv', index=False)
multi_Y_train.to_csv(folder + 'train/Y_train.csv', index=False)

multi_X_valid.to_csv(folder + 'valid/X_valid.csv', index=False)
multi_Y_valid.to_csv(folder + 'valid/Y_valid.csv', index=False)

multi_X_test.to_csv(folder + 'test/X_test.csv', index=False)
multi_Y_test.to_csv(folder + 'test/Y_test.csv', index=False)